In [1]:
%matplotlib inline
import utils; reload(utils)
from utils import *

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
!pwd

/home/ubuntu/courses/deeplearning1/lesson2


In [3]:
path = '../data/fishies'
train_path = path + '/train'
valid_path = path + '/valid'
test_path = path + '/test_stg1'
results_path = path + '/results'
batch_size=64

In [4]:
train_batches = get_batches(train_path, batch_size=64)
valid_batches = get_batches(valid_path, batch_size=64)
test_batches = get_batches(test_path, batch_size=64)

Found 3064 images belonging to 8 classes.
Found 713 images belonging to 8 classes.
Found 1000 images belonging to 1 classes.


In [5]:
import keras.layers.core
import keras.layers.convolutional
import keras.models
import keras.utils.data_utils
# Create vgg model
def vgg_preprocess(x):
    # Magic numbers that were the RGB averages of all Imagenet images
    mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))
    x = x - mean
    return x[:, ::-1] # reverse from RGB to BGR
model = keras.models.Sequential()
model.add(keras.layers.core.Lambda(vgg_preprocess, input_shape=(3,224,224)))

# VGG16 layer configuration.
for layers, filters in [(2,64), (2,128), (3,256), (3,512), (3,512)]:
    for i in range(layers):
        model.add(keras.layers.convolutional.ZeroPadding2D((1, 1)))
        model.add(keras.layers.convolutional.Convolution2D(filters, 3, 3, activation='relu'))
    model.add(keras.layers.convolutional.MaxPooling2D((2, 2), strides=(2, 2)))

# This just reshapes the outputs of previous nodes into a 1 dimensional thing?
# TODO
model.add(keras.layers.core.Flatten())

# Add 2 fully connected layers
# TODO why?
model.add(keras.layers.core.Dense(4096, activation='relu'))
model.add(keras.layers.core.Dense(4096, activation='relu'))

# Add a dropout layer, which randomly sets 50% of inputs to 0 at each update during training
# TODO why would you do this
model.add(keras.layers.core.Dropout(0.5))

# Add a final 1000 node layer for all 1000 Imagenet categories
model.add(Dense(1000, activation='softmax'))

# Load weights pre-trained by the vgg folks. Since we defined exactly the vgg architecture,
# we can reuse what they did before
model.load_weights(
    keras.utils.data_utils.get_file(
        'vgg16.h5',
        'http://www.platform.ai/models/vgg16.h5',
        cache_subdir='models'))

In [7]:
# By building the VGG16 model as it was before, we can use the pretrained weights.
# After loading the weight, we can tune it for our use case by removing the 1000-class detector
# and adding one with 8 layers, one for each fishy type
model.pop()
for layer in model.layers:
    layer.trainable = False
model.add(keras.layers.core.Dense(8,activation='softmax'))
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [8]:
# Train for 1 epoch
model.fit_generator(
    train_batches,
    samples_per_epoch=train_batches.nb_sample,
    nb_epoch=1,
    validation_data=valid_batches,
    nb_val_samples=valid_batches.nb_sample)

Epoch 1/1
3064/3064 [==============================] - 112s - loss: 1.9455 - acc: 0.4974 - val_loss: 0.8410 - val_acc: 0.7321


In [11]:
def onehot(batches):
    num_classes = len(batches.class_indices)
    return np.equal(batches.classes, np.matrix(np.arange(num_classes)).T).T.astype(int)
def multiclass_logloss_rowscaled(batches, preds):
    preds = np.clip(preds, 0.0001, 0.9999)
    sums = np.matrix(np.sum(preds, axis=1)).T
    preds = np.divide(preds, sums)
    oh = onehot(batches)
    return (-1.0/batches.N) * np.sum(np.multiply(oh, np.log(preds)))

In [16]:
# Run validation data through to get predictions
valid_preds = model.predict_generator(valid_batches, valid_batches.nb_sample)

In [17]:
multiclass_logloss_rowscaled(valid_batches, valid_preds)

3.3595376222890438

In [18]:
# Run test data through to get predictions
test_preds = model.predict_generator(test_batches, test_batches.nb_sample)

In [19]:
multiclass_logloss_rowscaled(test_batches, test_preds)

39.381199044999782

In [17]:
# Generate CSV
sorted_indices = sorted(train_batches.class_indices)
header = ','.join(['image'] + [i for i in sorted(train_batches.class_indices)])
with open(results_path + '/results.csv', 'w') as f:
    f.write(header + '\n')
    for filename, prow in zip(test_batches.filenames, preds):
        f.write(filename.replace('UNKNOWN/','') + ',' + ','.join(map(str, prow)) + '\n')